In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import *
from sklearn.linear_model import *
from sklearn.kernel_ridge import * 
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
ind = ['Season','Week','NeutralSite', 'Spread',  
  'HomePtsF', 'HomePtsA', 'HomeWins', 'HomeWinPct',
  'VisPtsF', 'VisPtsA', 'VisWins', 'VisWinPct',
  'HomePythPct', 'HomePythWins', 'HomeLuck', 
  'VisPythPct', 'VisPythWins', 'VisLuck', 
  'HomeElo', 'VisElo', 'HomeEloProb', 'VisEloProb', 'EloSpread', 'HomeConf_NotD1', 'VisConf_NotD1']

In [12]:
data_list = [0,0,0]

for i, year in  enumerate(range(2013, 2016)):
    data_list[i] = pd.read_csv('data/scores_pe/scores_pythElo{}.csv'.format(year))#[ind]


train = pd.concat(data_list)
train = train.loc[~((train['Season']==2013)&(train['Week']==4)),]
test = pd.read_csv('data/scores_pe/scores_pythElo{}.csv'.format(2016))#[ind]

In [7]:
from plotnine import *

In [5]:
preds = LinearRegression().fit(train.drop('Spread',axis=1), train[['Spread']]).predict(test.drop('Spread',axis=1))
mean_squared_error(test[['Spread']], preds)

315.55933158746666

In [15]:
def indices(date, ind):
    return pd.Series([date.year, date.month, date.day], index=ind)


def add_indices(data, ind):
    data[ind] = pd.to_datetime(data[ind]).dt.date
    data[['Year','Month','Day']] = data[ind].apply(lambda x: indices(x, ['Year','Month','Day']))
    data.drop(ind, axis=1, inplace=True)
    return data

odds = pd.read_csv('data/NCAAF_Odds.csv')
odds = add_indices(odds, 'DATE(date)')
odds = odds.rename({'Away':'Visiter'}, axis=1)
odds.drop(['HomeScore','AwayScore'],axis=1, inplace=True)

#oddsTrain = odds.loc[(odds['Month']<8)&(odds['Day']<27)&(odds['Year']<2016),:]
oddsTest = odds.loc[(odds['Month']>=8)&(odds['Day']>=27)&(odds['Year']>=2016),:]
#oddsTrain = pd.merge(train, odds, on=['Year','Month','Day','Home','Visiter'], how='left')

In [11]:
allScores2013 = pd.merge(scores2013, odds, on=['Year','Month','Day','Home','Visiter'], how='left')

Index(['Season', 'Week', 'NeutralSite', 'Spread', 'HomePtsF', 'HomePtsA',
       'HomeWins', 'HomeWinPct', 'VisPtsF', 'VisPtsA', 'VisWins', 'VisWinPct',
       'HomePythPct', 'HomePythWins', 'HomeLuck', 'VisPythPct', 'VisPythWins',
       'VisLuck', 'HomeElo', 'VisElo', 'HomeEloProb', 'VisEloProb',
       'EloSpread', 'HomeConf_NotD1', 'VisConf_NotD1'],
      dtype='object')

In [25]:
allScores = pd.concat([train,test])
allScores = pd.merge(allScores, odds, on=['Year','Month','Day','Home','Visiter'], how='left')
train = allScores.loc[allScores['Season']!=2016,:]
test = allScores.loc[allScores['Season']==2016,:]

In [27]:
preds = LinearRegression().fit(train.drop('Spread',axis=1), train[['Spread']]).predict(test.drop('Spread',axis=1))
mean_squared_error(test[['Spread']], preds)

,Season,Start,Visiter,VisID,Home,HomeID,NeutralSite,Home1Q,Home2Q,Home3Q,...,OverOdds_SBG,UnderOdds_SBG,Over_under_BetUS,Spread_BetUS,HomeML_BetUS,AwayML_BetUS,HomeSpreadOdds_BetUS,AwaySpreadOdds_BetUS,OverOdds_BetUS,UnderOdds_BetUS
11109,2016,2016-08-27,HAW,62,CAL,25,1,17.0,17.0,7.0,...,-110.0,-110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11110,2016,2016-08-27,CHSO,2127,NDSU,2449,0,0.0,3.0,7.0,...,-110.0,-110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11111,2016,2016-09-01,HUM,2278,CNEW,2105,0,7.0,21.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11112,2016,2016-09-01,EAS,2188,BRE,2913,0,0.0,7.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11113,2016,2016-09-01,LOC,209,CLA,2134,0,3.0,21.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11114,2016,2016-09-01,STA,2830,WAG,2681,0,3.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11115,2016,2016-09-01,MER,2385,ASH,308,0,14.0,21.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11116,2016,2016-09-01,VAW,2842,TUS,2839,0,0.0,3.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11117,2016,2016-09-01,COA,2557,ANN,15,0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11118,2016,2016-09-01,TULN,2655,WAKE,154,0,0.0,7.0,0.0,...,-110.0,-110.0,43.0,-14.5,-600.0,450.0,-110.0,-110.0,-110.0,-110.0
